In [114]:
import datetime, itertools, sys
import pandas as ps
from subprocess import Popen, PIPE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classification_report
from scipy.optimize import minimize
from pymystem3 import Mystem

In [115]:
data = ps.read_csv("data/spelled-f.csv", sep=';', header=None,
                   index_col=0,names=['id','title','text','cluster','date','publisher'])

## Предварительная очистка данных

In [116]:
data = data[~data["cluster"].isin(["-", "S", "Standard "])]
print("Число записей в таблице:", len(data))

Число записей в таблице: 32317


In [117]:
day_counts = {}
for _, row in data.iterrows():
    day_counts[row["date"]] = day_counts.get(row["date"], 0) + 1
max_date = max((count, date) for date, count in day_counts.items())[1]
print("Самый плотный день - {}, число записей: {}".format(max_date, day_counts[max_date]))

Самый плотный день - 2016-01-27, число записей: 19846


In [141]:
appropriate_days = [date for date, count in day_counts.items() if 520 <= count <= 530]
ch_day = appropriate_days[0]
data_1d = data[data["date"] == ch_day]
print("Выбранный день - {}, число записей: {}".format(ch_day, day_counts[ch_day]))

Выбранный день - 2015-05-30, число записей: 523


## Лемматизация текстов

In [143]:
m = Mystem()
cluster_index = {cluster: i for i, cluster in enumerate(data["cluster"].unique())}
messages = ["".join(m.lemmatize(row["title"] + ". " + row["text"])) for _, row in data_1d.iterrows()]
clusters = [cluster_index[row["cluster"]] for _, row in data_1d.iterrows()]

In [144]:
print(messages[0])
print(len(messages), len(clusters))

памятный комплекс «чкаловец — герой отечество» открывать в воронежский область. многий воронежец внимательно следить за соревнование летчик на полигон «погоновый». …«мы открывать памятник 262 герой советский союз, 12 дважды герой ссср и 13 герой россия - выпускник...

523 523


## Построение матрицы TF*IDF

In [145]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(messages)
print(tfidf_matrix.shape)

(523, 3849)


## Эксперимент с косинусовой мерой

In [146]:
index_cluster = {index: cluster for cluster, index in cluster_index.items()}
cluster_counts = {}
for cluster in clusters:
    cluster_counts[cluster] = cluster_counts.get(cluster, 0) + 1
max_cluster = max((count, cluster) for cluster, count in cluster_counts.items())[1]
print("Самый большой кластер - '{}' ({})\nЧисленность - {}".format(
        index_cluster[max_cluster], max_cluster, cluster_counts[max_cluster]))

Самый большой кластер - 'В Тюмени нарушитель угнал машину ДПС, пока оформлялся протокол' (1706)
Численность - 3


In [147]:
print("Все сообщения кластера:\n")
for i, cluster in enumerate(clusters):
    if cluster == max_cluster:
        print("  " + messages[i])

Все сообщения кластера:

  в тюмень пассажир останавливать «лад» во время составление протокол «угнать» машина. в тот момент, когда инспектор ДПС оформлять протокол задержание на водитель останавливать «лад», ее пассажир, воспользоваться момент, попытаться скрываться на отечественный машина. …в тот...

  в тюмень нарушитель угнать машина ДПС, пока силовик составлять протокол. силовик останавливать тонировать «лад» 13-ой модель. в салон сидеть трое пьяный молодой человек, который препятствовать составление протокол.

  угон полицейский машина в тюмень оказываться «утка». представитель гибдд тюменский область опровергать информация об угон автомобиль ДПС во время составление административный протокол. сегодня утро на улица ватутин в тюмень наряд ДПС быть...



In [148]:
target_messages = [i for i, cluster in enumerate(clusters) if cluster == max_cluster]
ind = target_messages[0]
sim_factors = cosine_similarity(tfidf_matrix[ind:ind+1], tfidf_matrix)
for i in target_messages:
    print(sim_factors[0][i])

1.0
0.385132473118
0.261807268993


In [149]:
th_values = (0.8, 0.6, 0.4, 0.35, 0.3, 0.25, 0.2)
thresholds = {t: 0 for t in th_values}
for sf in sim_factors[0]:
    for th in th_values:
        if sf >= th:
            thresholds[th] += 1
print(sorted(thresholds.items()))

[(0.2, 3), (0.25, 3), (0.3, 2), (0.35, 2), (0.4, 1), (0.6, 1), (0.8, 1)]


## Эксперимент на полноразмерных данных

In [150]:
sim_factors_full = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [127]:
def get_expected_and_predicted(sim_factors, clusters, th=0.25):
    #print("get_expected_and_predicted:", len(sim_factors), len(clusters))
    res = {"pair": [], "y": [], "p": []}
    for i1, i2 in itertools.combinations(range(len(sim_factors)), 2):
        res["pair"].append(sorted((i1, i2)))
        res["y"].append(clusters[i1] == clusters[i2])
        res["p"].append(sim_factors[i1][i2] >= th)
    return ps.DataFrame(res, index=None)

In [128]:
def cross_class_report(res):
    classes = res["y"].unique()
    table = ps.DataFrame(index=classes, columns=classes)
    for true_cls in classes:
        tmp = res[res["y"] == true_cls]
        for pred_cls in classes:
            table[pred_cls][true_cls] = len(tmp[tmp["p"] == pred_cls])
    return table

In [129]:
def get_prec_recall_f(res, cls):
    pred = res[res["p"] == cls]
    prec = len(pred[pred["y"] == cls]) / len(pred)
    act = res[res["y"] == cls]
    rec = len(act[act["p"] == cls]) / len(act)
    return prec, rec, 2/(1/prec + 1/rec)

In [130]:
def get_performance_for_threshold(sim_factors, clusters, th):
    res = get_expected_and_predicted(sim_factors, clusters, th)
    return get_prec_recall_f(res, 1)

In [162]:
def optimise_threshold(sim_factors, clusters):
    #print("optimise_threshold", len(sim_factors), len(clusters))
    f = lambda th: get_performance_for_threshold(sim_factors, clusters, th)[2]
    fx = {}
    for i in range(1, 9):
        x = i*0.1
        try:
            fx[x] = f(x)
        except ZeroDivisionError:
            fx[x] = 0
    best = sorted(((val, x) for x, val in fx.items()), reverse=True)
    x1, x2 = sorted((best[0][1], best[1][1]))
    return max((f(x*0.01), x*0.01) for x in range(int(x1*100), int(x2*100) + 1))[1]

In [163]:
th = optimise_threshold(sim_factors_full, clusters)
res = get_expected_and_predicted(sim_factors_full, clusters, th)
print("*** THRESHOLD: {} ***".format(th))
print(classification_report(res["y"], res["p"]))
print(cross_class_report(res), "\n\n")

*** THRESHOLD: 0.31 ***
             precision    recall  f1-score   support

      False       1.00      1.00      1.00    136121
       True       0.47      0.69      0.56       382

avg / total       1.00      1.00      1.00    136503

        False True 
False  135830   291
True      119   263 




## Проверка на нескольких выборках

In [164]:
print(sorted((count, day) for day, count in day_counts.items()))

[(2, '2015-01-22'), (11, '2015-10-29'), (15, '2015-11-29'), (17, '2015-01-23'), (17, '2015-08-28'), (18, '2015-09-29'), (22, '2015-01-24'), (22, '2015-08-25'), (24, '2015-04-30'), (25, '2015-09-26'), (28, '2015-10-26'), (28, '2015-11-28'), (38, '2015-10-30'), (41, '2015-09-30'), (48, '2015-03-30'), (51, '2015-09-28'), (75, '2015-03-28'), (81, '2015-12-23'), (82, '2015-03-26'), (87, '2015-11-27'), (94, '2015-11-30'), (102, '2015-08-30'), (104, '2016-02-06'), (111, '2015-08-29'), (118, '2015-11-26'), (119, '2015-09-27'), (121, '2015-11-25'), (144, '2015-10-25'), (164, '2015-06-26'), (217, '2015-06-25'), (226, '2015-05-27'), (236, '2015-05-28'), (240, '2015-03-25'), (259, '2015-05-25'), (262, '2015-02-27'), (279, '2015-02-25'), (281, '2015-05-29'), (288, '2015-02-26'), (315, '2015-05-26'), (393, '2015-03-29'), (523, '2015-05-30'), (536, '2015-02-28'), (1189, '2016-01-28'), (1865, '2016-01-26'), (3553, '2016-02-05'), (19846, '2016-01-27')]


In [165]:
def get_data_for_day(day):
    data_1d = data[data["date"] == day]
    
    cluster_index = {cluster: i for i, cluster in enumerate(data_1d["cluster"].unique())}
    messages = ["".join(m.lemmatize(row["title"] + ". " + row["text"])) for _, row in data_1d.iterrows()]
    clusters = [cluster_index[row["cluster"]] for _, row in data_1d.iterrows()]
    
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(messages)
    sim_factors = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
    return sim_factors, clusters

In [166]:
days_to_check = ['2015-11-29', '2015-09-26', '2015-09-28', '2015-03-28', '2015-08-30',
                '2015-10-25', '2015-06-25', '2015-05-25', '2015-05-26', '2015-05-30']
scores = []
for day in days_to_check:
    sim_factors, clusters = get_data_for_day(day)
    th = optimise_threshold(sim_factors, clusters)
    res = get_expected_and_predicted(sim_factors, clusters, th)
    prec, recall, f = get_prec_recall_f(rems, 1)
    scores.append((prec, recall, f, day_counts[day]))
    print("День {}, число сообщений: {}".format(day, day_counts[day]))
    print("Порог: {}".format(th))
    print(classification_report(res["y"], res["p"]))
    print(cross_class_report(res), "\n\n")

День 2015-11-29, число сообщений: 15
Порог: 0.3
             precision    recall  f1-score   support

      False       0.97      1.00      0.98        96
       True       1.00      0.67      0.80         9

avg / total       0.97      0.97      0.97       105

      False True 
False    96     0
True      3     6 


День 2015-09-26, число сообщений: 25
Порог: 0.18
             precision    recall  f1-score   support

      False       1.00      1.00      1.00       283
       True       0.94      1.00      0.97        17

avg / total       1.00      1.00      1.00       300

      False True 
False   282     1
True      0    17 


День 2015-09-28, число сообщений: 51
Порог: 0.21
             precision    recall  f1-score   support

      False       0.99      1.00      1.00      1236
       True       0.86      0.82      0.84        39

avg / total       0.99      0.99      0.99      1275

      False True 
False  1231     5
True      7    32 


День 2015-03-28, число сообщений: 75
П

In [167]:
total_pairs = sum(s[3]*(s[3]-1)//2 for s in scores)
avg_prec = sum(prec*(count*(count-1)//2)/total_pairs for prec, _, _, count in scores)
avg_rec = sum(rec*(count*(count-1)//2)/total_pairs for _, rec, _, count in scores)
avg_f = sum(f*(count*(count-1)//2)/total_pairs for _, _, f, count in scores)
print("Средняя точность:", avg_prec)
print("Средняя полнота:", avg_rec)
print("Средняя Ф-мера:", avg_f)

Средняя точность: 0.6059880104001494
Средняя полнота: 0.7156058808271637
Средняя Ф-мера: 0.6488098181498925


In [94]:
print(scores)

[(1.0, 0.3333333333333333, 0.25, 15), (1.0, 0.23529411764705882, 0.19047619047619047, 25), (1.0, 0.5897435897435898, 0.3709677419354839, 51), (0.9322033898305084, 0.873015873015873, 0.45081967213114754, 75), (0.7047619047619048, 0.8809523809523809, 0.39153439153439157, 102), (0.9555555555555556, 0.37719298245614036, 0.2704402515723271, 144), (0.7608695652173914, 0.8536585365853658, 0.40229885057471265, 217), (0.9022556390977443, 0.6451612903225806, 0.3761755485893417, 259), (0.697560975609756, 0.6355555555555555, 0.33255813953488367, 315), (0.4363905325443787, 0.7722513089005235, 0.27882797731569, 523)]
